In [ ]:
%pip install llama-index-llms-openai
!pip install llama-index
#import nest_asyncio

#nest_asyncio.apply()
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.WARN)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


Note: you may need to restart the kernel to use updated packages.


In [83]:
from llama_index.core import SimpleDirectoryReader

# documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()



In [84]:
from llama_index.core.node_parser import SentenceSplitter

nodes = SentenceSplitter().get_nodes_from_documents(documents)


In [85]:
nodes

[TextNode(id_='0e1ae618-7579-4904-9f03-31e7d2424018', embedding=None, metadata={'page_label': '1', 'file_name': '2022 Q3 AAPL.pdf', 'file_path': '..\\data\\2022 Q3 AAPL.pdf', 'file_type': 'application/pdf', 'file_size': 266240, 'creation_date': '2024-11-13', 'last_modified_date': '2024-11-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0fc2abe8-449c-4738-9717-b171fe095128', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': '2022 Q3 AAPL.pdf', 'file_path': '..\\data\\2022 Q3 AAPL.pdf', 'file_type': 'application/pdf', 'file_size': 266240, 'creation_date': '2024-11-13', 'last_modified_date': '2024-11-13'}, hash='f45eecf247de3241092057fdfd5d5ace98a54671e118515122c35a16394bbc6

In [86]:
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)


In [87]:
len(docstore.docs)


29

In [88]:
docstore.docs.keys()


dict_keys(['0e1ae618-7579-4904-9f03-31e7d2424018', '1c311b18-b9d9-4776-8793-98f6223eece4', 'b488f3d3-1310-4642-875e-71992879b382', '02313c97-d72e-44ed-bf96-fc55a9a8b09c', 'a14993ec-d6cc-49a2-ac56-cb9422144f88', '2bbe6264-af4e-4e01-886c-dd6ba0a1f5d4', '2011811c-c0dc-4005-b3de-752662f5d49b', 'f9df2ec2-8180-4bf1-99de-e2c02139503f', '81cf7083-6d8a-4947-9ada-2a54f5e4eecb', '9ee74393-f0b7-4c1b-b1d0-cf090f4b82d0', '1ad1fecb-563c-4eca-ace1-f63dd8ef054b', '5c914a14-218d-41b1-9591-7af082ee8cae', 'da7fe643-1a38-4160-9bbb-d6ff6a0c5133', '4bdbdfdc-b5c5-4ed6-a325-a5569f3bb017', '757421eb-e736-47dc-8c96-3b99cff3b2c8', '49e285b8-f820-4317-89b6-ecd37208bf7b', '349d846c-558a-4521-bdb5-306703ae49d8', '7f1ba38b-cd38-413e-86a2-11775429094c', 'f2269bcc-a4e9-4c72-8878-33c7b27659cc', '34738082-c337-4f00-aacf-b11cb380a813', '218ebef1-1547-4efc-b343-9f76ed7e155f', '84f41f76-67e5-485c-8553-63af56610d7b', 'b4de287f-26ff-4eb5-ba22-5a35673f5227', '8998ee93-7327-44c9-ba82-aa23a1249d67', '41db123a-2acc-4347-b815-5f1c

In [89]:
doc_key = list(docstore.docs.keys())[0]  # Select the first document key
docstore.docs[doc_key]


TextNode(id_='0e1ae618-7579-4904-9f03-31e7d2424018', embedding=None, metadata={'page_label': '1', 'file_name': '2022 Q3 AAPL.pdf', 'file_path': '..\\data\\2022 Q3 AAPL.pdf', 'file_type': 'application/pdf', 'file_size': 266240, 'creation_date': '2024-11-13', 'last_modified_date': '2024-11-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0fc2abe8-449c-4738-9717-b171fe095128', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': '2022 Q3 AAPL.pdf', 'file_path': '..\\data\\2022 Q3 AAPL.pdf', 'file_type': 'application/pdf', 'file_size': 266240, 'creation_date': '2024-11-13', 'last_modified_date': '2024-11-13'}, hash='f45eecf247de3241092057fdfd5d5ace98a54671e118515122c35a16394bbc6b

In [90]:
node = docstore.docs[doc_key]
print(node.text)  # Displays the text content of the node


UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-Q
(Mark One)
☒  QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the quarterly period ended June 25, 2022
or
☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from              to             .
Commission File Number: 001-36743
Apple Inc.
(Exact name of Registrant as specified in its charter)
California 94-2404110
(State or other jurisdictionof incorporation or organization) (I.R.S. Employer Identification No.)
One Apple Park Way
Cupertino, California 95014
(Address of principal executive offices) (Zip Code)
(408) 996-1010
(Registrant’s telephone number, including area code)
Securities registered pursuant to Section 12(b) of the Act:
Title of each class Trading symbol(s) Name of each exchange on which registered
Common Stock, $0.00001 par value per shareAAPL The Nasdaq Stock Market LLC
1.000% Notes

In [91]:
#os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")
from llama_index.core import Settings
from llama_index.llms.ollama import  Ollama
Settings.llm = Ollama(model='llama3.2:latest', base_url='http://localhost:11434',temperature=0.1)



In [92]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",  # Replace with your desired model
    base_url="http://localhost:11434",  # Ensure Ollama is running at this endpoint
    ollama_additional_kwargs={"mirostat": 0} #Mirostat is a technique for controlling perplexity and balancing the text generation process in large language models (LLMs).
) 

In [93]:
from llama_index.core import StorageContext, SummaryIndex, VectorStoreIndex, SimpleKeywordTableIndex

storage_context = StorageContext.from_defaults(docstore=docstore)
summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=ollama_embedding)
keyword_table_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTT

In [94]:
query_engine = summary_index.as_query_engine()
response = query_engine.query("What this document is about?")
print(response)




INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:H

In [95]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:114

In [96]:
query_engine = keyword_table_index.as_query_engine()
response = query_engine.query("Which company annual or quarterly report is this?")
print(response)

INFO:llama_index.core.indices.keyword_table.retrievers:> Starting query: Which company annual or quarterly report is this?
> Starting query: Which company annual or quarterly report is this?
> Starting query: Which company annual or quarterly report is this?
> Starting query: Which company annual or quarterly report is this?
> Starting query: Which company annual or quarterly report is this?
> Starting query: Which company annual or quarterly report is this?
> Starting query: Which company annual or quarterly report is this?
INFO:llama_index.core.indices.keyword_table.retrievers:query keywords: ['annual', 'company', 'quarterly', 'report']
query keywords: ['annual', 'company', 'quarterly', 'report']
query keywords: ['annual', 'company', 'quarterly', 'report']
query keywords: ['annual', 'company', 'quarterly', 'report']
query keywords: ['annual', 'company', 'quarterly', 'report']
query keywords: ['annual', 'company', 'quarterly', 'report']
query keywords: ['annual', 'company', 'quarterly

In [97]:
len(docstore.docs)

29

In [98]:
from llama_index.core import Document  # Import the Document class

# Example: Adding new documents
new_documents = [
    Document(text="This is a new document."),
    Document(text="Here is another document to add."),
]

# Convert documents into nodes
new_nodes = SentenceSplitter().get_nodes_from_documents(new_documents)


docstore.add_documents(new_nodes)

# Verify the contents of the DocumentStore
print(f"Total documents in the docstore: {len(docstore.docs)}")
for key, node in docstore.docs.items():
    print(f"Key: {key}, Text: {node.text}")


Total documents in the docstore: 31
Key: 0e1ae618-7579-4904-9f03-31e7d2424018, Text: UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-Q
(Mark One)
☒  QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the quarterly period ended June 25, 2022
or
☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from              to             .
Commission File Number: 001-36743
Apple Inc.
(Exact name of Registrant as specified in its charter)
California 94-2404110
(State or other jurisdictionof incorporation or organization) (I.R.S. Employer Identification No.)
One Apple Park Way
Cupertino, California 95014
(Address of principal executive offices) (Zip Code)
(408) 996-1010
(Registrant’s telephone number, including area code)
Securities registered pursuant to Section 12(b) of the Act:
Title of each class Trading symbol(s) Name of each exchange on which registered
Co

In [99]:
len(docstore.docs)

31

In [100]:
from llama_index.core.schema import TextNode

# Example: Fetch a node to update
doc_key = list(docstore.docs.keys())[0]  # Get the first document key
node = docstore.docs[doc_key]

# Create a new node with updated metadata
updated_node = TextNode(
    text="Text Updated to test Node text update",  # Keep the original text
    extra_info={"source": "Updated Source"}  # Update the metadata
)

# Remove the old node and add the updated node
docstore.delete_document(doc_key)
docstore.add_documents([updated_node])

# Verify the updated node
print(f"Updated Node Metadata: {updated_node.extra_info}")


Updated Node Metadata: {'source': 'Updated Source'}


In [101]:
len(docstore.docs)

31

In [102]:
doc_key = list(docstore.docs.keys())[30]  # Select the first document key
docstore.docs[doc_key]
node = docstore.docs[doc_key]
print(node.text)  # Displays the text content of the node


Text Updated to test Node text update


In [106]:

# Example: Replacing all the documents
# Wrap plain strings in `Document` objects
new_documents = [
    Document(text="This is a new document."),
    Document(text="Here is another document to add."),
]

# Convert documents into nodes
new_nodes = SentenceSplitter().get_nodes_from_documents(new_documents)

# Create a new DocumentStore and add the new nodes
new_docstore = SimpleDocumentStore()
new_docstore.add_documents(new_nodes)

# Verify the contents of the new DocumentStore
print(f"Total documents in the new docstore: {len(new_docstore.docs)}")
for key, node in new_docstore.docs.items():
    print(f"Key: {key}, Text: {node.text}")

docstore=new_docstore    


Total documents in the new docstore: 2
Key: 214b5197-5a02-465c-bc47-ceb461b4bb4c, Text: This is a new document.
Key: 4541a7a2-896f-4e5f-b94d-7522e3a064cd, Text: Here is another document to add.


In [107]:
len(docstore.docs)

2